In [15]:
from src import utils
import numpy as np
import os
import librosa
from joblib import Parallel, delayed
import pandas as pd

In [2]:
corp = utils.CorpusMaker.from_excel('corpus_updated', only_30_corpus=False).tracks

In [7]:
proc_path = f'{utils.get_project_root()}/data/processed/mvsep_audio'
raw_path = f'{utils.get_project_root()}/data/raw/audio'

In [44]:
def checker(fpath):
    procfiles = [i for i in os.listdir(proc_path) if fpath in i]
    rawfiles = [i for i in os.listdir(raw_path) if fpath in i]
    bigaudio = sum(librosa.load(os.path.join(proc_path, f), sr=utils.SAMPLE_RATE, mono=True, dtype=np.float64, duration=20, offset=10)[0] for f in procfiles)
    rawaudio = sum(librosa.load(os.path.join(raw_path, f), sr=utils.SAMPLE_RATE, mono=True, dtype=np.float64, duration=20, offset=10)[0] for f in rawfiles)
    # rawaudio = librosa.load(os.path.join(raw_path, fpath + '.wav'), sr=utils.SAMPLE_RATE, mono=True, dtype=np.float64, duration=10)[0]
    assert bigaudio.shape == rawaudio.shape
    return fpath, np.corrcoef(bigaudio, rawaudio)[0, 1]

In [45]:
with Parallel(n_jobs=-1, backend='loky', verbose=10) as par:
    res = par(delayed(checker)(fp['fname']) for fp in corp)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:  

In [46]:
df = pd.DataFrame(res).rename(columns={0: 'track', 1:'r'})

In [47]:
df[df['r'] < 0.8]['track'].values

array(['jonesh-swonderful-hintonmrosengardenb-1977-d3373c2a',
       'jamala-raincheck-crosbyifournierv-1960-30cd5578',
       'jamala-speaklow-crosbyifournierv-1960-566dfdda',
       'evansb-swedishpastry-israelscbunkerl-1963-36012ad1',
       'evansb-gloriasstep-gomezemorellm-1972-b519428f',
       'evansb-upwiththelark-johnsonmlabarberaj-1980-a2bb66b1',
       'evansb-likesomeoneinlove-johnsonmlabarberaj-1980-2f3f1a5e',
       'evansb-thetwolonelypeople-johnsonmlabarberaj-1980-e5016fe2',
       'evansb-daysofwineandroses-johnsonmlabarberaj-1980-964c123f',
       'evansb-likesomeoneilove-johnsonmlabarberaj-1980-62b3c57e',
       'evansb-lettertoevan-johnsonmlabarberaj-1980-d9ac22be',
       'evansb-autumnleaves-johnsonmlabarberaj-1980-2955522b',
       'petersono-liza-brownrthigpene-1959-fd4117b5',
       'garlandr-sosorryplease-jonesspersipc-1961-f42fcd79'], dtype=object)

In [48]:
df['r'].describe()

count    1313.000000
mean        0.970371
std         0.046837
min         0.690320
25%         0.949006
50%         0.998857
75%         0.999531
max         0.999936
Name: r, dtype: float64